In [22]:
from ib_insync import *
import pandas as pd
from datetime import datetime, timedelta
import requests
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, MultiHeadAttention, LayerNormalization, Add, Flatten

# Inicializar conexión

In [23]:
util.startLoop()

In [24]:
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)

<IB connected to 127.0.0.1:7497 clientId=1>

Datos de la divisa a extraer

In [25]:
# Definir el contrato de GBP/USD
contract = Forex('GBPUSD')

# Establecer el rango de tiempo
end_time = datetime.now()
start_time = end_time - timedelta(days=90)

# Obtener datos históricos
bars = ib.reqHistoricalData(
    contract,
    endDateTime=end_time,
    durationStr='90 D',
    barSizeSetting='1 hour',
    whatToShow='MIDPOINT',
    useRTH=False
)

# Convertir los datos a DataFrame
df = util.df(bars)

Conexion con la api de las noticias

In [26]:
# Clave de API de News API (regístrate para obtener una clave)
API_KEY = 'b35c56d955ee45178c703f7f79c1dfca'

In [27]:

# Obtener noticias recientes relacionadas con GBP/USD (últimas 24 horas)
news_url = f'https://newsapi.org/v2/everything?q=GBP%20USD&from={(end_time - timedelta(days=1)).strftime("%Y-%m-%dT%H:%M:%SZ")}&sortBy=publishedAt&apiKey={API_KEY}'
response = requests.get(news_url)


In [28]:
# Parsear la respuesta JSON
news_data = response.json()

# Extraer titulares relevantes
titles = [article['title'] for article in news_data['articles']]
print(f"Titulares de noticias relevantes:\n {titles}")


Titulares de noticias relevantes:
 ["Retro-Style Platformer 'Ninja 1987' Isn't Shy About Its Inspirations"]


In [29]:
# Contar palabras clave positivas y negativas
positive_keywords = ['increase', 'growth', 'positive']
negative_keywords = ['decline', 'decrease', 'negative']

positive_news = sum(1 for title in titles if any(word in title.lower() for word in positive_keywords))
negative_news = sum(1 for title in titles if any(word in title.lower() for word in negative_keywords))

In [30]:
# Seleccionar las columnas relevantes
data = df[['open', 'high', 'low', 'close']].values

In [31]:
# Normalizar los datos
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)


In [32]:
# Ajustar los datos con el impacto de las noticias
scaled_data_adjusted = scaled_data * impact_factor


In [33]:
# Crear secuencias de entrenamiento para Transformers
sequence_length = 60
x_train, y_train = [], []

for i in range(sequence_length, len(scaled_data_adjusted)):
    x_train.append(scaled_data_adjusted[i-sequence_length:i])
    y_train.append(scaled_data_adjusted[i])

x_train, y_train = np.array(x_train), np.array(y_train)

Uso del algoritmo k-means

In [34]:
# Implementar K-Means para encontrar patrones ocultos
kmeans = KMeans(n_clusters=4)
clusters = kmeans.fit_predict(scaled_data_adjusted)

# Agregar las etiquetas de los clusters a los datos
scaled_data_adjusted = np.hstack([scaled_data_adjusted, clusters.reshape(-1, 1)])


In [35]:
# Actualizar las secuencias de entrada con las etiquetas de cluster
x_train = []
for i in range(sequence_length, len(scaled_data_adjusted)):
    x_train.append(scaled_data_adjusted[i-sequence_length:i])
x_train = np.array(x_train)

In [36]:
# Construir el modelo Transformers
input_layer = Input(shape=(x_train.shape[1], x_train.shape[2]))
attention = MultiHeadAttention(num_heads=4, key_dim=4)(input_layer, input_layer)
attention = LayerNormalization()(attention + input_layer)

flatten = Flatten()(attention)
dense1 = Dense(50, activation='relu')(flatten)
dropout = Dropout(0.2)(dense1)
output_layer = Dense(4)(dropout)  # Predicción de open, high, low, close

model = Model(inputs=input_layer, outputs=output_layer)

# Compilar el modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el modelo
model.fit(x_train, y_train, epochs=20, batch_size=32)


Epoch 1/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.5570
Epoch 2/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0498
Epoch 3/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0390
Epoch 4/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0322
Epoch 5/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0303
Epoch 6/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0282
Epoch 7/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0254
Epoch 8/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0228
Epoch 9/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0188
Epoch 10/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0175
Epoch 11/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0175
Epoch 12/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0159
Epoch 13/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0132
Epoch 14/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0152
Epoch 15/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0119
Epoch 16/20
65/65 ━

In [37]:
# Obtener datos recientes para predecir
recent_data = scaled_data_adjusted[-sequence_length:, :-1]

# Lista para almacenar las predicciones

In [60]:
# Predecir desde las 10 p.m. hasta la 1 p.m. (16 horas)
for _ in range(16):
    # Asegurar que recent_data tenga forma (60, 5)
    if recent_data.shape[1] != 8:
        raise ValueError(f"Expected recent_data to have 5 columns, but got {recent_data.shape[1]}")
    
    # Preparar el input con la forma adecuada para el modelo
    x_input = np.expand_dims(recent_data, axis=0)  # Dar forma (1, 60, 5)
    
    # Realizar la predicción
    predicted_price_normalized = model.predict(x_input, verbose=0)  # Salida (1, 4)
    
    # Desnormalizar la predicción
    predicted_price = scaler.inverse_transform(
        np.hstack([predicted_price_normalized, np.zeros((predicted_price_normalized.shape[0], 1))])
    )[:, :4]  # Solo 'open', 'high', 'low', 'close'
    
    # Guardar la predicción desnormalizada
    predictions.append(predicted_price[0])  # Extraer la fila
    
    # Normalizar nuevamente para actualizar recent_data
    predicted_price_scaled = scaler.transform(
        np.hstack([predicted_price, [[0]]])  # Agregar dummy cluster para mantener 5 columnas
    )
    
    # Actualizar recent_data desplazando hacia la izquierda
    recent_data = np.append(recent_data[1:], [predicted_price_scaled[0]], axis=0)

# Crear un DataFrame con las predicciones
prediction_times = [end_time.replace(hour=17, minute=0, second=0) + timedelta(hours=i) for i in range(1, 17)]
predicted_df = pd.DataFrame(predictions, columns=['open', 'high', 'low', 'close'], index=prediction_times)

print(predicted_df)

InvalidArgumentError: Graph execution error:

Detected at node functional_2_1/multi_head_attention_2_1/key_1/MatMul defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\alex_\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "C:\Users\alex_\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\alex_\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\Users\alex_\AppData\Roaming\Python\Python312\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "c:\Users\alex_\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py", line 641, in run_forever

  File "C:\Users\alex_\AppData\Roaming\Python\Python312\site-packages\nest_asyncio.py", line 133, in _run_once

  File "c:\Users\alex_\Ap